# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

weather_df = pd.read_csv("/Users/rorosun/Desktop/Data Analytics Monash University/python-api-challenge/WeatherPy/WeatherPy.csv", encoding="utf-8")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,port alfred,-33.5906,26.8910,57.38,75,86,6.35,ZA,1655686374
1,rikitea,-23.1203,-134.9692,72.07,66,91,22.82,PF,1655686375
2,atar,20.5169,-13.0499,80.22,63,95,5.12,MR,1655686375
3,iqaluit,63.7506,-68.5145,51.53,46,75,17.27,CA,1655686376
4,hermanus,-34.4187,19.2345,43.81,84,0,11.10,ZA,1655686377


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_weather_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0), :]
narrowed_weather_df = narrowed_weather_df.dropna(how='any')
narrowed_weather_df.reset_index(inplace=True)
del narrowed_weather_df['index']
narrowed_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mantua,45.1667,10.7833,76.15,77,0,3.00,IT,1655686380
1,san lawrenz,36.0553,14.2040,71.85,86,0,6.55,MT,1655686411
2,asyut,27.1810,31.1837,77.18,36,0,12.33,EG,1655686418
3,hohhot,40.8106,111.6522,79.09,36,0,4.47,CN,1655686433
4,alta floresta,-9.8756,-56.0861,71.42,83,0,0.00,BR,1655686434


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotellist = []

for i in range(len(narrowed_weather_df)):
    lat = narrowed_weather_df.loc[i]['Lat']
    lng = narrowed_weather_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_weather_df["Hotel Name"] = hotellist
narrowed_weather_df = narrowed_weather_df.dropna(how='any')
narrowed_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,mantua,45.1667,10.7833,76.15,77,0,3.00,IT,1655686380,Mantua
1,san lawrenz,36.0553,14.2040,71.85,86,0,6.55,MT,1655686411,Victoria
2,asyut,27.1810,31.1837,77.18,36,0,12.33,EG,1655686418,Asyut
3,hohhot,40.8106,111.6522,79.09,36,0,4.47,CN,1655686433,Hohhot
4,alta floresta,-9.8756,-56.0861,71.42,83,0,0.00,BR,1655686434,Alta Floresta


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_weather_df.iterrows()]
locations = narrowed_weather_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))